## load 参数

In [27]:
import numpy as np
import cv2
import scipy.io as scio

Params=scio.loadmat('workspace.mat')
K = np.array(Params['K'])
K = K.T
distortion=np.array(Params['distortion'])

## sift特征匹配

In [28]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

imgpath = 'images/'
MIN_MATCH_COUNT = 10

def match_pts(imgname1, imgname2):
    img1 = cv2.imread(imgpath+imgname1, 0)          # queryImage
    img2 = cv2.imread(imgpath+imgname2, 0)        # trainImage

    # Initiate SIFT detector
    sift = cv2.xfeatures2d.SIFT_create()

    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)

    FLANN_INDEX_KDTREE = 0
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks = 50)

    flann = cv2.FlannBasedMatcher(index_params, search_params)

    matches = flann.knnMatch(des1,des2,k=2)

    # store all the good matches as per Lowe's ratio test.
    good = []
    pts1 = []
    pts2 = []
    
    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.8*n.distance:
            good.append(m)
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)

    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)
    # return the np.array type
    return pts1, pts2

# estimate估计参数

In [29]:
from numpy import *
from scipy import *
from scipy.linalg import *
from pylab import *
import cv2
import sys

def find_fundamental(pts1, pts2):
    F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_LMEDS)
    return F
    
def find_essential(F, K1, K2):
    return K2.T @ F @ K1

def skew(a):
    #Skew matrix A such that cross(a,v) = Av for any v
    return array([[0,-a[2],a[1]],[a[2],0,-a[0]],[-a[1],a[0],0]])

def anti_skew(A):
    return array([A[2,1], A[2,0], A[1,0]])

def triangulate(P1, P2, pts1, pts2):
    n1 = pts1.shape[0]
    n2 = pts2.shape[0]
    
    p1 = vstack([ pts1.T, ones(n1) ]).T
    p2 = vstack([ pts2.T, ones(n2) ]).T
    
    P = zeros((n1, 4))
    for i in range(n1):
        cross1 = skew(p1[i])
        cross2 = skew(p2[i])
        Q = vstack([cross1 @ P1, cross2 @ P2])
        U,S,VT = svd(Q)
        z = VT[-1, :]
        P[i, :] = z/z[3]
    
    return P[:, 0:3]

def guess_rotation(E):
    # make sure E is rank 2
    U,S,V = svd(E)
    if det(dot(U,V)) < 0:
        V = -V
    E = dot(U,dot(diag([1,1,0]),V))    

    W = array([[0,-1,0],[1,0,0],[0,0,1]])
        
    R21 = dot(U,dot(W,V))
    R22 = dot(U,dot(W.T,V))
    
    T21 = anti_skew(E @ R21.T) 
    T22 = anti_skew(E @ R22.T)
    
    return R21, T21, R22, T22

def test_rotation(P1, P2, pts1, pts2):
    #temp = cv2.triangulatePoints(P1, P2, transpose(pts1), transpose(pts2))
    temp = triangulate(P1, P2, pts1, pts2)
    for v in temp:
        if v[-1] < 0:
            return False
    return True

def errnum_rotation(P1, P2, pts1, pts2):
    #temp = cv2.triangulatePoints(P1, P2, transpose(pts1), transpose(pts2))
    temp = triangulate(P1, P2, pts1, pts2)
    num = 0
    for v in temp:
        if v[-1] < 0:
            num += 1
    return num

def find_rotation(E, K1, K2, pts1, pts2):
    R1 = eye(3)
    T1 = array([0, 0, 0])
    P1 = K1 @ vstack([R1.T, T1]).T
    
    R21, T21, R22, T22 = guess_rotation(E)
    
    P21 = K2 @ vstack([R21.T, T21]).T
    P22 = K2 @ vstack([R22.T, T22]).T

    #key1 = test_rotation(P1, P21, pts1, pts2)
    #key2 = test_rotation(P1, P22, pts1, pts2)
    
    
    errn1 = errnum_rotation(P1, P21, pts1, pts2)
    errn2 = errnum_rotation(P1, P22, pts1, pts2)
    
    R2 = R21
    T2 = T21

    if errn1 < errn2:
        R2 = R21
        T2 = T21
    elif errn2 < errn1:
        R2 = R22
        T2 = T22
    else:
        print('Can not find the right R matrix.')
        sys.exit()

    return R1, T1, R2, T2
    
def find_pose(R1, T1, R2, T2, K1, K2):
    P1 = K1 @ vstack([R1.T, T1]).T
    P2 = K2 @ vstack([R2.T, T2]).T
    
    return P1, P2


## 运行

In [30]:
pts1, pts2 = match_pts('1.jpg', '2.jpg')
print(pts1.shape)
F = find_fundamental(pts1, pts2)
E = find_essential(F, K, K)
R1, T1, R2, T2 = find_rotation(E, K, K, pts1, pts2)

print(R1, T1, R2, T2, sep='\n')
K1 = K
K2 = K
P1, P2 = find_pose(R1, T1, R2, T2, K1, K2)
points = triangulate(P1, P2, pts1, pts2)

(683, 2)
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
[0 0 0]
[[ 0.78399147  0.11398967  0.61021614]
 [ 0.10804562 -0.99304898  0.04668916]
 [ 0.61129659  0.02932727 -0.79085801]]
[-0.95967927  0.03935584 -0.27832862]


In [31]:
#print(K, F, E, sep='\n')
print(points[100:1010, ])

[[-0.14274509  0.97119732  1.94636403]
 [ 1.15145261  0.25785249  0.03718155]
 [ 0.53287348 -0.18771554  1.28735497]
 ...
 [ 0.31410651  0.43721357  0.66284998]
 [ 0.33503535  0.31914735  0.65717057]
 [ 0.34988361  0.09882088  0.87205219]]


## MVS多图

In [32]:
def sparse(imgname1, imgname2, K1, K2):
    pts1, pts2 = match_pts(imgname1, imgname2)
    F = find_fundamental(pts1, pts2)
    E = find_essential(F, K1, K2)
    
    R1, T1, R2, T2 = find_rotation(E, K1, K2, pts1, pts2)
    
    P1, P2 = find_pose(R1, T1, R2, T2, K1, K2)
    
    X = triangulate(P1, P2, pts1, pts2)
    R = R2
    T = T2
    
    return X, R, T

def multi_sparse(imgs, K):
    n = len(imgs)
    xn = n-1
    Xs = []
    Rs = []
    Ts = []
    for i in range(xn):
        Xtemp, Rtemp, Ttemp = sparse(imgs[i], imgs[i+1], K, K)
        print(i, Xtemp[10:20,])
        print(Rtemp, Ttemp)
        
        Xs.append(Xtemp)
        Rs.append(Rtemp)
        Ts.append(Ttemp)
    
    X = Xs[0]

    for i in range(1, xn):
        newX = Xs[i]
        nn = newX.shape[0]
        newX = newX.T
        for j in range(1, i + 1):
            newX = vstack([Rs[i-j].T, Ts[i-j]]).T @ vstack([newX, ones(nn)])
        newX = newX.T
        X = vstack([X, newX])
        
    return X
    
imgs = ['1_.jpg', '2_.jpg', '3_.jpg', '4_.jpg', '5_.jpg', '6_.jpg']
X = multi_sparse(imgs, K)
print(X.shape)

0 [[-0.35661128 -0.43659305  0.93034978]
 [-0.34301304 -0.34980693  0.92813531]
 [-0.33386419  0.39211506  0.92167546]
 [-0.34851695 -0.29648506  0.90495577]
 [-0.34077716 -0.31871689  0.93038593]
 [-0.33400331  0.40028925  0.92735175]
 [-0.33617337  0.16014765  0.93541626]
 [-0.32150419  0.25339911  0.90837155]
 [-0.32938769  0.13424337  0.93105858]
 [-0.34082633 -0.38378825  0.95494034]]
[[ 9.33062652e-01 -3.86475291e-03 -3.59693134e-01]
 [-3.86090148e-03 -9.99992281e-01  7.29121780e-04]
 [-3.59693175e-01  7.08423451e-04 -9.33070371e-01]] [ 0.9751416  -0.00491469 -0.22152812]


error: OpenCV(3.4.2) C:\projects\opencv-python\opencv_contrib\modules\xfeatures2d\src\sift.cpp:1121: error: (-5:Bad argument) image is empty or has incorrect depth (!=CV_8U) in function 'cv::xfeatures2d::SIFT_Impl::detectAndCompute'
